# Walmart API overview
##### Search API used for look ups. We will use this first
##### Taxonomy API used for classifications.  We want to find a way to count classifications

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import urllib
import json
import sys
import pprint
import os
import random

# We are using 2 api Keys for redundancy and because
# there is a limit to the number of calls
api_key = "4tsbjxvbrwnpcjdsh277csqu"
api_alt = "anrtcd5dheh9758z5wgdy677"

# Base url for all API look ups
base_url = "http://api.walmartlabs.com/v1/" 

# Variables to specify the API
search = "search?"
paginated_products = "paginated/items?"

# Base url for search
search_url = base_url + search
paginated_url = base_url + paginated_products



# Search API

In [6]:
# Load search criteria for Walmart Search API.
# reference URL: https://developer.walmartlabs.com/docs/read/Search_API

format = "json"
categoryId = "3944"
query = "laptop"
numItems = 25 # max 25. 10 is default
start = 1
sort = "price"
order = "desc"
facet = "on" # boolean
min = str(0) # lower end of range for search within price range
max = str(200) # higher end of range price for search with price range
facet_range = (f"price:[{min}%20TO%20{max}]") # note the format [num1:num2] does not work. Use %20TO%20 for ":"



# NOTE: we are using 2 methods to create urls. Because the params argument does not format the output
# in the manner needed to get the API to work.  This is used only for price range lookups

# manual url. Used for price range lookup only
manual_url = (f"{search_url}apiKey={api_alt}&format={format}&categoryId={categoryId}&query={query}&numItems={numItems}&facet={facet}&facet.range={facet_range}")

print(manual_url)

# API parameters for other lookups
params = {
    "apiKey": api_alt,
    #"start": start, # note keep turned off.  We can't get this to work yet
    "format": "json", # json|xml
    "categoryId": categoryId,
    "query": query,
    "numItems": numItems,
    #"sort": sort, #allowed sort types are [relevance, price, title, bestseller, customerRating, new]
    #"order": order,  #allowed values are [asc, desc]
    #"facet":facet, # note keep turned off.  We can't get this to work yet
    #"facet.range":facet_range    # note keep turned off.  We can't get this to work yet
}

http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[0%20TO%20200]


In [7]:
# Loop by pricing parameters to pull out entire data set for a product category
# to save time we are going to hard code some numbers then go back later if we have time

# Number of items found in our search
# there are 20,747 total results for laptop in electronics
# step 0 there are 3089 results in the price range $0-$200
# step 1 there are 3015 results in the price range $201-$400
# step 2 there are 2053 results in the price range $401-$600
# step 3 there are 2995 results in the price range $601-$1000
# step 4 there are 2021 results in the price range $1001-$1400
# step 5 there are 2726 results in the price range $1401-$1800
# step 6 there are 1873 results in the price range $1801-$2200
# step 7 there are 1011 results in the price range $2201-$2600
# step 8 there are 863 results in the price range $2601-3000
# step 9 there are 1096 results in the price range above $3000

category_total = 20747 #hard coded from broad search
step0 = 200*25//3089
step1 = 200*25//3015
step2 = 200*25//2053
step3 = 400*25//2995
step4 = 400*25//2021
step5 = 400*25//2726
step6 = 400*25//1873
step7 = 400*25//1011
step8 = 400*25//863

# create a function to perform the price search

def price_data(lowRange, highRange):
    facet_range = (f"price:[{lowRange}%20TO%20{highRange}]")
    price_url = (f"{search_url}apiKey={api_alt}&format={format}&categoryId={categoryId}&query={query}&numItems={numItems}&facet={facet}&facet.range={facet_range}")
    walmart_data2 = requests.get(price_url)
    walmart_data2 = walmart_data2.json()
    print (price_url)
    try:       
        a = []
        if not os.path.isfile('Walmart_datasets/walmart_laptops.json'):
            a.append(walmart_data2['items'])
            with open('Walmart_datasets/walmart_laptops.json', mode='w') as f:
                f.write(json.dumps(walmart_data2['items'], indent=4))
        else:
            with open('Walmart_datasets/walmart_laptops.json') as feedsjson:
                feeds = json.load(feedsjson)
            
            feeds.append(walmart_data2['items'])
            with open('Walmart_datasets/walmart_laptops.json', mode='w') as f:
                f.write(json.dumps(feeds, indent=4))

    except:
        print ("Skipped line")
                


In [10]:
# Use for loop to go through prices in the steps defined

searchCount = 0
itemCount = 0
sleepCount = 0
low = 3000
high = 3000

for itemCount in range (1,12000):
    searchCount += 1
    itemCount = 25*searchCount
    print(f"search / item count = {searchCount} / {itemCount}")
    
    # We have had issues pulling all the data.  Walmart could be shutting us down
    # We will add some random sleep every 100 queries to try to trick the servers
    sleepCount += 1
    if sleepCount == 100:
        sleepTime = random.randint(60, 120)
        time.sleep(sleepTime)
        print(f"resting for {sleepTime} seconds")
        sleepCount = 0
    elif high < 200:
        high = low + step0
        price_data(low,high)
        low = high
    elif high < 400:
        high = low + step1
        price_data(low,high)
        low = high
    elif high < 600:
        high = low + step2
        price_data(low,high)
        low = high
    elif high < 1000:
        high = low + step3
        price_data(low,high)
        low = high      
    elif high < 1400:
        high = low + step4
        price_data(low,high)
        low = high
    elif high < 1800:
        high = low + step5
        price_data(low,high)
        low = high
    elif high < 2200:
        high = low + step6
        price_data(low,high)
        low = high    
    elif high < 2600:
        high = low + step7
        price_data(low,high)
        low = high
    elif high < 3000:
        high = low + step8
        price_data(low,high)
        low = high
    elif high < 8000:
        high = low + step8
        price_data(low,high)
        low = high
    else:
        print ('finished')
        print (f'item count = {itemCount}')
        print (f'final search count = {searchCount}')
        print (f'final high price is ${high}')
        break
               
      



search / item count = 1 / 25
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[3000%20TO%203011]
search / item count = 2 / 50
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[3011%20TO%203022]
search / item count = 3 / 75
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[3022%20TO%203033]
Skipped line
search / item count = 4 / 100
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[3033%20TO%203044]
search / item count = 5 / 125
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[3044%20TO%203055]


search / item count = 42 / 1050
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[3451%20TO%203462]
Skipped line
search / item count = 43 / 1075
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[3462%20TO%203473]
search / item count = 44 / 1100
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[3473%20TO%203484]
search / item count = 45 / 1125
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[3484%20TO%203495]
search / item count = 46 / 1150
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[3495%

search / item count = 83 / 2075
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[3902%20TO%203913]
search / item count = 84 / 2100
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[3913%20TO%203924]
Skipped line
search / item count = 85 / 2125
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[3924%20TO%203935]
search / item count = 86 / 2150
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[3935%20TO%203946]
search / item count = 87 / 2175
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[3946%

Skipped line
search / item count = 124 / 3100
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[4342%20TO%204353]
search / item count = 125 / 3125
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[4353%20TO%204364]
search / item count = 126 / 3150
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[4364%20TO%204375]
Skipped line
search / item count = 127 / 3175
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[4375%20TO%204386]
Skipped line
search / item count = 128 / 3200
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&fa

Skipped line
search / item count = 164 / 4100
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[4782%20TO%204793]
Skipped line
search / item count = 165 / 4125
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[4793%20TO%204804]
search / item count = 166 / 4150
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[4804%20TO%204815]
Skipped line
search / item count = 167 / 4175
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[4815%20TO%204826]
Skipped line
search / item count = 168 / 4200
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&n

...

Skipped line
search / item count = 282 / 7050
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[6069%20TO%206080]
Skipped line
search / item count = 283 / 7075
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[6080%20TO%206091]
search / item count = 284 / 7100
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[6091%20TO%206102]
Skipped line
search / item count = 285 / 7125
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[6102%20TO%206113]
Skipped line
search / item count = 286 / 7150
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&n

Skipped line
search / item count = 322 / 8050
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[6498%20TO%206509]
search / item count = 323 / 8075
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[6509%20TO%206520]
Skipped line
search / item count = 324 / 8100
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[6520%20TO%206531]
Skipped line
search / item count = 325 / 8125
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[6531%20TO%206542]
Skipped line
search / item count = 326 / 8150
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&n

Skipped line
search / item count = 361 / 9025
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[6927%20TO%206938]
Skipped line
search / item count = 362 / 9050
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[6938%20TO%206949]
Skipped line
search / item count = 363 / 9075
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[6949%20TO%206960]
Skipped line
search / item count = 364 / 9100
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[6960%20TO%206971]
Skipped line
search / item count = 365 / 9125
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&q

Skipped line
search / item count = 400 / 10000
resting for 82 seconds
search / item count = 401 / 10025
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[7356%20TO%207367]
Skipped line
search / item count = 402 / 10050
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[7367%20TO%207378]
Skipped line
search / item count = 403 / 10075
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[7378%20TO%207389]
Skipped line
search / item count = 404 / 10100
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[7389%20TO%207400]
Skipped line
search / item count = 405 / 10125
http://api.walmartlabs.com/v1/search

http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[7774%20TO%207785]
Skipped line
search / item count = 440 / 11000
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[7785%20TO%207796]
Skipped line
search / item count = 441 / 11025
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[7796%20TO%207807]
Skipped line
search / item count = 442 / 11050
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[7807%20TO%207818]
Skipped line
search / item count = 443 / 11075
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.ran

In [ ]:
from pprint import pprint
import os
filepath = os.path.join('Walmart_datasets/walmart_laptops.json')
with open(filepath) as jsonfile:
    json_data = json.load(jsonfile)
    print (len(json_data))
    for i in range(len(json_data)):
        try:
            print (json_data['itemId'])
        except:
            print(f"skipped entry {i}")
    
